In [1]:
import pandas as pd
import os
import random

file_names = os.listdir("twitter-data")
tweets = pd.DataFrame()
tweets_col_names = ["time_stamp", "user", "col1", "col2", "col3", "tweet"]

for file in file_names:
    if file.endswith("tsv"):
        print(file)
        tweets = tweets.append(pd.read_csv("twitter-data/" + file, sep="\t", encoding="utf-8", names=tweets_col_names))

tweets = tweets.head()
tweets

twitter_jpm_1561481394.tsv
twitter_jpm_1561481711.tsv
twitter_jpm_1561482054.tsv
twitter_jpm_1561482382.tsv
twitter_jpm_1561482725.tsv
twitter_jpm_1561483051.tsv
twitter_jpm_1561483374.tsv
twitter_jpm_1561483719.tsv
twitter_jpm_1561484061.tsv
twitter_jpm_1561484432.tsv
twitter_jpm_1561484782.tsv
twitter_jpm_1561485104.tsv
twitter_jpm_1561485484.tsv
twitter_jpm_1561485828.tsv
twitter_jpm_1561486163.tsv


,time_stamp,user,col1,col2,col3,tweet
0,1012437258497155072,BarrySchwartzBW,1.530219e+09,1.0,12,$JPM raises its dividend 43%
1,1012455046922022912,CNBC,1.530223e+09,5.0,6,"After-hours buzz: NKE, WFC, SNX, KBH, JPM, C, ..."
2,1012407195269394432,danprimack,1.530212e+09,21.0,111,"Amazon may or may not put HQ2 in Boston, but B..."
3,1012660746700902400,psk2329,1.530272e+09,NaN,7,$BAC $C $JPM got bids yesterday plus this good...
4,1012063014059560961,jpmorgan,1.530130e+09,19.0,74,Congratulations to $JPM Chief Marketing Office...


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jeremy.seow.2014\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
#placeholder method => using Random number
def getSentiment(body):
#     body = clean_tweets(body)
    sent_list = [0,2,4]
    return (random).choice(sent_list)

def getTweetData(min_date, rolling_window):
    
    #   convert date str
    #     target_date = "2015-01-01"
    tweets['created'] = pd.to_datetime(tweets['time_stamp'], format='%Y/%m/%d',errors='coerce') 
    tweets['sentiment'] = "1"
    temp = tweets[tweets['created'] > min_date]
        
    for row in temp.itertuples():
        temp.loc[row.Index, 'created'] = row.created.date()
        temp.loc[row.Index, 'sentiment'] = getSentiment(row.without)

    #group by date and sentiment and unstack
    df = temp.groupby(['created','sentiment']).size().unstack(level=-1).fillna(0)
    #fill in missing time series with 0 values
    df = df.resample('D').mean().fillna(0)
    #generate rolling window, sort reverse
    df = df.rolling(rolling_window).sum()[::-1]
    
    return df

In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re #regular expression
import string

stop_words = set(stopwords.words('english'))

#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

emoticons = emoticons_happy.union(emoticons_sad)

def clean_tweets(tweet):
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
    #remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
    
    return ' '.join([word for word in tweet.split() if word not in (stop_words) and word not in emoticons and word not in string.punctuation])

def check_eng_char(word):
    try:
        word.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True
    
tweets = tweets[tweets['tweet'].apply(lambda y: check_eng_char(y))]
tweets["without"] = tweets['tweet'].apply(lambda x: clean_tweets(x))

tweets

,time_stamp,user,col1,col2,col3,tweet,without
0,1012437258497155072,BarrySchwartzBW,1.530219e+09,1.0,12,$JPM raises its dividend 43%,$JPM raises dividend 43%
2,1012407195269394432,danprimack,1.530212e+09,21.0,111,"Amazon may or may not put HQ2 in Boston, but B...","Amazon may may put HQ2 Boston, Boston clearly ..."
3,1012660746700902400,psk2329,1.530272e+09,NaN,7,$BAC $C $JPM got bids yesterday plus this good...,$BAC $C $JPM got bids yesterday plus good news...
4,1012063014059560961,jpmorgan,1.530130e+09,19.0,74,Congratulations to $JPM Chief Marketing Office...,"Congratulations $JPM Chief Marketing Officer, ..."


In [19]:
getTweetData('1950-01-01', 2)

""
